# **Libraries**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Commented out IPython magic to ensure Python compatibility.
# %matplotlib inline
import warnings
warnings.filterwarnings("ignore")
from scipy import sparse
from scipy.sparse import hstack
import pandas as pd
import numpy as np
import nltk
import math
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from sklearn.feature_extraction.text import CountVectorizer
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from IPython.display import Image
import string
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle
import math
from lightgbm import LGBMRegressor
from tqdm import tqdm
import os
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input , Dropout, Flatten,concatenate,LSTM
from tensorflow.keras.layers import Embedding
import tensorflow.keras
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ModelCheckpoint,TensorBoard,ReduceLROnPlateau, EarlyStopping
from tensorflow.keras import backend as K
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input , Dropout, Flatten,concatenate,LSTM

# **Data Preprocessing**

In [3]:
def data_cleaning(df):

  df['category_name']=np.where(df['category_name'].isnull(),'Unknown_Category',df['category_name'])
  df['brand_name']=np.where(df['brand_name'].isnull(),'Unknown_Brand',df['brand_name'])
  df['item_description']=np.where(df['item_description'].isnull(),'Unknown_description',df['item_description'])

  df3 =df['category_name'].str.split('/',expand=True)
  df3.columns = ['category{}'.format(x+1) for x in df3.columns]
  l=df3.columns.tolist()
  while len(l)!=5:
    l.append('category{}'.format(len(l)+1))
    
  for j in l:
    for i in df3.columns:
      if j not in df3:
        df3[j]=np.nan
    
  df = df.join(df3)  

  df['category1']=np.where(df['category1'].isnull(),'undefined',df['category1'])
  df['category2']=np.where(df['category2'].isnull(),'undefined',df['category2'])
  df['category3']=np.where(df['category3'].isnull(),'undefined',df['category3'])
  df['category4']=np.where(df['category4'].isnull(),'undefined',df['category4'])
  df['category5']=np.where(df['category5'].isnull(),'undefined',df['category5'])

  return df

In [4]:
# https://stackoverflow.com/a/47091490/4084039
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [5]:
# https://gist.github.com/sebleier/554280
# we are removing the words from the stop words list: 'no', 'nor', 'not'
stopwords= ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"]

In [6]:
def preprocess_txt(data):
    preprocessed = []
    # tqdm is for printing the status bar
    for sentance in data:
        sent = decontracted(sentance)
        sent = sent.replace('\\r', ' ')
        sent = sent.replace('\\"', ' ')
        sent = sent.replace('\\n', ' ')
        sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
        # https://gist.github.com/sebleier/554280
        sent = ' '.join(e for e in sent.split() if e not in stopwords)
        preprocessed.append(sent.lower().strip())
    return preprocessed

In [7]:
def basic_text_features(df,column,stopwords):

  df[column]=df[column].astype(str)

  df["No_of_words"+column]=df[column].apply(lambda x : len(str(x).split(" ")))
  df["No_of_chars"+column]=df[column].str.len()
  df['Avg_len_'+column]=df[column].apply(lambda x : (sum(len(word) for word in x.split())/len(x.split())))
  df["No_of_stop_words"+column]=df[column].apply(lambda x: len([x for x in x.split() if x in stopwords]))
  df['No_of_digits'+column]=df[column].apply(lambda x : len([x for x in x.split() if x.isdigit()]))
  df['No_of_upper_cased'+column]=df[column].apply(lambda x : len([x for x in x.split() if x.isupper()]))
  df['No_of_lower_cased'+column]=df[column].apply(lambda x : len([x for x in x.split() if x.islower()]))

  return df


In [8]:
def encoding_categorical_variables(col,X_train,X_test,df):
  
  """ One hot Encode Categorical Variables using countvectorizer """
  
  from collections import Counter
  my_counter = Counter()
  for word in df[col].astype(str).values:
      my_counter.update(word.split())

  cat_dict = dict(my_counter)
  sorted_cat_dict = dict(sorted(cat_dict.items(), key=lambda kv: kv[1]))

  from sklearn.feature_extraction.text import CountVectorizer
  vectorizer = CountVectorizer(vocabulary=list(sorted_cat_dict.keys()), lowercase=False, binary=True)
  X_train_cat = vectorizer.transform(X_train[col].values)
  X_test_cat = vectorizer.transform(X_test[col].values)
  
  return X_train_cat ,X_test_cat

In [9]:
def scaling_numerical_features(col,df_train,df_test):
  
  """ Normalizin Numerical Variables """

  from sklearn.preprocessing import Normalizer
  normalizer = Normalizer()
  normalizer.fit(df_train[col].values.reshape(-1,1))

  train_norm = normalizer.transform(df_train[col].values.reshape(-1,1))
  test_norm = normalizer.transform(df_test[col].values.reshape(-1,1))

  return train_norm , test_norm

In [10]:
def encoding_numerical_categories(col,X_train,X_test):
  """ Encoding Numerical Categories """

  from sklearn.preprocessing import OneHotEncoder
  enc = OneHotEncoder()
  enc.fit(X_train[col].values.reshape(-1, 1))
  X_train_enc = enc.transform(X_train[col].values.reshape(-1, 1))
  X_test_enc = enc.transform(X_test[col].values.reshape(-1, 1))

  return X_train_enc,X_test_enc

# **Define functions for Models**

In [11]:
def mlp_model3(dim):

  Input_layer=Input(shape=(dim.shape[1],))
  Input_lay=Dense(2046, activation='relu')(Input_layer)
  L1=Dense(1024, activation='relu')(Input_lay)
  L2=Dense(512, activation='relu')(L1)
  L3=Dense(256, activation='relu')(L2)
  L4=Dense(128, activation='relu')(L3)
  L5=Dense(64, activation='relu')(L4)
  L6=Dense(32, activation='relu')(L5)
  out=Dense(1)(L6)
  model=Model(Input_layer,out)

  return model

In [12]:
def mlp_model4(dim):
 
  Input_layer=Input(shape=(dim.shape[1],))
  #Input_lay=Dense(2046, activation='relu')(Input_layer)
  L1=Dense(1024, activation='relu')(Input_layer)
  L2=Dense(512, activation='relu')(L1)
  L3=Dense(256, activation='relu')(L2)
  L4=Dense(128, activation='relu')(L3)
  L5=Dense(64, activation='relu')(L4)
  L6=Dense(32, activation='relu')(L5)
  out=Dense(1)(L6)
  model=Model(Input_layer,out)

  return model

In [13]:
def MLP_Models(mlp3,mlp4,X_te_tfidf):
  """ Get predictions fromMLP Models """
  model = mlp3
  model.load_weights('/content/drive/My Drive/Kaggle/Mercari/approach_1/DL_Models/model-001-0.163283-0.163283.h5')
  preds = model.predict(X_te_tfidf,batch_size = 512,verbose = 1)[:, 0]
  preds_mlp3=np.expm1((preds.reshape(-1, 1))[:, 0])

  model = mlp4
  model.load_weights('/content/drive/My Drive/Kaggle/Mercari/approach_1/DL_Models/model-001-0.162473-0.162473.h5')
  preds = model.predict(X_te_tfidf,batch_size = 200,verbose = 1)[:, 0]
  preds_mlp4=np.expm1((preds.reshape(-1, 1))[:, 0])

  pred_final_mlp=(0.510204*preds_mlp4) + ((1-0.510204)*preds_mlp3)

  return pred_final_mlp

In [14]:
def LGBM_models(X_te_tfidf):
  LGBM_1 = pickle.load(open('/content/drive/My Drive/Kaggle/Mercari/approach_1/DL_Models/LGBM_best15000.pkl',"rb"))
  preds_LGBM_1=np.expm1(LGBM_1.predict(X_te_tfidf))
  LGBM_2 = pickle.load(open('/content/drive/My Drive/Kaggle/Mercari/approach_1/DL_Models/LGBM_best10000.pkl',"rb"))
  preds_LGBM_2=np.expm1(LGBM_2.predict(X_te_tfidf))

  pred_final_LGBM=(1.000000*preds_LGBM_1) + ((1-1.000000)*preds_LGBM_2)

  return pred_final_LGBM

# **Function 1**

In [19]:
def function1(X):

  ########################### Data Cleaning #######################################

  X_inp= data_cleaning(X)
  X_inp['product_name_preprocessed']=preprocess_txt(X_inp['name'])
  X_inp['item_description_preprocessed']=preprocess_txt(X_inp['item_description'].astype(str))
  stop=stopwords
  X_inp=basic_text_features(X_inp,"item_description",stop)
  X_inp=basic_text_features(X_inp,"name",stop)
  X_test=X_inp

  X= pd.read_pickle('/content/drive/My Drive/Kaggle/Mercari/approach_1/X.pkl')
  X_train=pd.read_pickle('/content/drive/My Drive/Kaggle/Mercari/approach_1/X_train.pkl')

  #################### Preprocessing and Encoding ###################################################
  X_train_brand_name,X_test_brand_name=encoding_categorical_variables('brand_name',X_train,X_test,X)
  X_train_category1,X_test_category1=encoding_categorical_variables('category1',X_train,X_test,X)
  X_train_category2,X_test_category2=encoding_categorical_variables('category2',X_train,X_test,X)
  X_train_category3,X_test_category3=encoding_categorical_variables('category3',X_train,X_test,X)
  X_train_category4,X_test_category4=encoding_categorical_variables('category4',X_train,X_test,X)
  X_train_category5,X_test_category5=encoding_categorical_variables('category5',X_train,X_test,X)

  

  X_train_wordsitem_description,X_test_wordsitem_description=scaling_numerical_features('No_of_wordsitem_description',X_train,X_test)
  X_train_charsitem_description,X_test_charsitem_description=scaling_numerical_features('No_of_charsitem_description',X_train,X_test)
  X_train_len_item_description,X_test_len_item_description=scaling_numerical_features('Avg_len_item_description',X_train,X_test)
  X_train_stop_wordsitem_description,X_test_stop_wordsitem_description=scaling_numerical_features('No_of_stop_wordsitem_description',X_train,X_test)
  X_train_digitsitem_description,X_test_digitsitem_description=scaling_numerical_features('No_of_digitsitem_description',X_train,X_test)
  X_train_upper_caseditem_description,X_test_upper_caseditem_description=scaling_numerical_features('No_of_upper_caseditem_description',X_train,X_test)
  X_train_lower_caseditem_description,X_test_lower_caseditem_description=scaling_numerical_features('No_of_lower_caseditem_description',X_train,X_test)

  X_train_wordsname,X_test_wordsname=scaling_numerical_features('No_of_wordsname',X_train,X_test)
  X_train_charsname,X_test_charsname=scaling_numerical_features('No_of_charsname',X_train,X_test)
  X_train_len_name,X_test_len_name=scaling_numerical_features('Avg_len_name',X_train,X_test)
  X_train_stop_wordsname,X_test_stop_wordsname=scaling_numerical_features('No_of_stop_wordsname',X_train,X_test)
  X_train_digitsname,X_test_digitsname=scaling_numerical_features('No_of_digitsname',X_train,X_test)
  X_train_upper_casedname,X_test_upper_casedname=scaling_numerical_features('No_of_upper_casedname',X_train,X_test)
  X_train_lower_casedname,X_test_upper_lower_casedname=scaling_numerical_features('No_of_lower_casedname',X_train,X_test)
  X_train_enc_shipping,X_test_enc_shipping = encoding_numerical_categories('shipping',X_train,X_test)
  X_train_enc_item_condition,X_test_enc_item_condition = encoding_numerical_categories('item_condition_id',X_train,X_test)

  ############################## Vectorization ##############################################################################
  vectorizer = TfidfVectorizer(max_features=100000)
  vectorizer.fit(X_train['product_name_preprocessed'].values.astype('U')) 

  X_train_prod_name_tfidf = vectorizer.transform(X_train['product_name_preprocessed'].values.astype('U'))
  X_test_prod_name_tfidf = vectorizer.transform(X_test['product_name_preprocessed'].values.astype('U'))

  vectorizer = TfidfVectorizer(max_features=100000)
  vectorizer.fit(X_train['item_description_preprocessed'].values.astype('U')) 

  X_train_item_desc_tfidf = vectorizer.transform(X_train['item_description_preprocessed'].values.astype('U'))
  X_test_item_desc_tfidf = vectorizer.transform(X_test['item_description_preprocessed'].values.astype('U'))

  ############################# Final Train Test Data #############################################################################
  X_tr_tfidf = hstack((X_train_prod_name_tfidf,X_train_item_desc_tfidf,X_train_brand_name,X_train_category1,X_train_category2,X_train_category3,X_train_category4,X_train_category5,X_train_wordsitem_description,X_train_charsitem_description,	X_train_len_item_description,X_train_stop_wordsitem_description,X_train_digitsitem_description,X_train_upper_caseditem_description,X_train_lower_caseditem_description,X_train_wordsname,X_train_charsname,X_train_len_name,X_train_stop_wordsname,X_train_digitsname,X_train_upper_casedname,X_train_lower_casedname,X_train_enc_shipping,X_train_enc_item_condition)).tocsr()
  X_te_tfidf = hstack((X_test_prod_name_tfidf,X_test_item_desc_tfidf,X_test_brand_name,X_test_category1,X_test_category2,X_test_category3,X_test_category4,X_test_category5,X_test_charsitem_description,X_test_len_item_description,X_test_stop_wordsitem_description,X_test_digitsitem_description,X_test_upper_caseditem_description,X_test_lower_caseditem_description,X_test_wordsitem_description,X_test_wordsname,X_test_charsname,X_test_len_name,X_test_stop_wordsname,X_test_digitsname,X_test_upper_casedname,X_test_upper_lower_casedname,X_test_enc_shipping,X_test_enc_item_condition)).tocsr()


  ################################## Prediction using trained Models #############################################################
  model3=mlp_model3(X_tr_tfidf)
  model4=mlp_model4(X_tr_tfidf)

  pred_final_mlp=MLP_Models(model3,model4,X_te_tfidf)
  pred_final_LGBM=LGBM_models(X_te_tfidf)
  

  pred_final_ensemble=(0.367347*pred_final_LGBM) + ((1-0.367347)*pred_final_mlp)

  return pred_final_ensemble

In [ ]:
df=pd.read_csv('/content/drive/My Drive/Kaggle/Mercari/test_stg2.tsv',sep='\t', header=0)
X_Input=df[2:3]
X_Input.set_index('test_id',inplace=True)

In [ ]:
%%time
pred=function1(X_Input)
print('The predicted price for product --- >',X_Input.values,'is--->',pred)

1/1 [==============================] - 0s 3ms/step
The predicted price for product --- > [['Coach bag' 1 'Vintage & Collectibles/Bags and Purses/Handbag' 'Coach'
  1 'Brand new coach bag. Bought for [rm] at a Coach outlet.']] is---> [60.11525188]
CPU times: user 1min 46s, sys: 5.75 s, total: 1min 51s
Wall time: 2min 56s


# **Function2**

In [15]:
def rmsle_score(y, y_pred):
    assert len(y) == len(y_pred)
    to_sum = [(math.log(y_pred[i] + 1) - math.log(y[i] + 1)) ** 2.0 for i,pred in enumerate(y_pred)]
    return (sum(to_sum) * (1.0/len(y))) ** 0.5

In [16]:
def function2(X,Y):
  score=rmsle_score(Y,function1(X))
  return score

In [21]:
df=pd.read_csv('/content/drive/My Drive/Kaggle/Mercari/test_stg2.tsv',sep='\t', header=0)
X_Input=df[2:3]
price=[70]


In [22]:
%%time
score=function2(X_Input,price)
print('The score --- >is',score)

1/1 [==============================] - 0s 2ms/step
The score --- >is 0.14991823311764385
CPU times: user 1min 47s, sys: 6.81 s, total: 1min 54s
Wall time: 2min 41s


In [26]:
df=pd.read_pickle('/content/drive/My Drive/Kaggle/Mercari/approach_1/X_test.pkl')
df.drop(['category1', 'category2', 'category3', 'category4', 'category5'], axis=1, inplace=True)
X_Input=df[2:3]
price=X_Input['price'].tolist()


In [27]:
%%time
score=function2(X_Input,price)
print('The score --- >is',score)

1/1 [==============================] - 0s 1ms/step
The score --- >is 0.1837494319053734
CPU times: user 1min 49s, sys: 4.6 s, total: 1min 54s
Wall time: 1min 53s
